In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np

In [2]:
# get data
csi300 = pd.read_csv("../data/AINDEXHS300CLOSEWEIGHT.csv")
industrial_info = pd.read_csv("../data/SWIndustry.csv")
industrial_name = pd.read_excel('../data/申万行业分类.xlsx')

today_Date = 20211203
# change the column name
csi300 = csi300[['S_INFO_WINDCODE','S_CON_WINDCODE','TRADE_DT','I_WEIGHT']]
industrial_info = industrial_info[['S_INFO_WINDCODE','SW_IND_CODE','ENTRY_DT','REMOVE_DT']]
industrial_info.columns = ['S_CON_WINDCODE','SW_IND_CODE','ENTRY_DT','REMOVE_DT']
csi300 = pd.merge(csi300,industrial_info,on='S_CON_WINDCODE',how='left')

# merge the dataframe 
csi300 = csi300.fillna(today_Date)
csi300['SW_INDEX'] = np.nan
csi_industrial = csi300[(csi300['TRADE_DT']>=csi300['ENTRY_DT'])&(csi300['TRADE_DT']<=csi300['REMOVE_DT'])]
csi300.loc[list(csi_industrial.index),'SW_INDEX'] = csi_industrial['SW_IND_CODE'].apply(lambda x:int(str(x)[:4]))
csi300 = csi300.dropna()

# rank the sw sector weight
industrial_sector1 = industrial_name[['一级行业代码','SW一级行业']].dropna()
industrial_sector1.columns = ['SW_INDEX','SectorCode']

industrial_weight = csi300.groupby('SW_INDEX').sum()
industrial_weight = industrial_weight.sort_values(by='I_WEIGHT',ascending= False)

industrial_lar5 = industrial_weight.iloc[:5]
industrial_lar5['SectorWeightRank'] = [1,2,3,4,5]

# choose the largest five weight sector and standardlize
csi300_lar5 = pd.merge(csi300,industrial_lar5,how='left',on='SW_INDEX')
csi300_lar5 = csi300_lar5.dropna()
csi300_lar5 = csi300_lar5[['S_CON_WINDCODE','SW_INDEX','SectorWeightRank','I_WEIGHT_x','I_WEIGHT_y']]
csi300_lar5 = pd.merge(csi300_lar5,industrial_sector1,on='SW_INDEX',how='left')
csi300_lar5 = csi300_lar5[['S_CON_WINDCODE','SectorCode','SectorWeightRank','I_WEIGHT_x','I_WEIGHT_y']]
csi300_lar5.columns = ['StockCode','SectorCode','SectorWeightRank','WeightInIndex','WeightInSectorPortofolio']
csi300_lar5['WeightInSectorPortofolio'] = csi300_lar5['WeightInIndex']/csi300_lar5['WeightInSectorPortofolio']
csi300_lar5.to_csv('../data/problem1.csv',encoding='utf_8_sig')

/Users/cescqi/PycharmProjects/venvCollection/phbs/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
